In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
prompt = hub.pull("lawwu/chain_of_density")

# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.1)
    | json_parser
)

In [4]:
chain.invoke(content)

[{'Missing_Entities': 'Optuna',
  'Denser_Summary': "This article discusses Optuna, a hyperparameter optimization framework. Optuna is a powerful tool for optimizing machine learning models. It provides various methods for suggesting hyperparameters, such as suggest_categorical, suggest_int, and suggest_float. With Optuna, you can easily optimize your model's performance by finding the best hyperparameters."},
 {'Missing_Entities': 'study',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization framework. Optuna provides a study object that allows you to optimize your model by finding the best hyperparameters. You can create a study and use the optimize() method to search for the optimal hyperparameters. The study keeps track of the best parameters found so far and allows you to access them using the best_params attribute.'},
 {'Missing_Entities': 'objective function',
  'Denser_Summary': 'This article discusses Optuna, a hyperparameter optimization framework

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json


# Load some data to summarize
loader = WebBaseLoader(
    "https://m.sports.naver.com/esports/article/311/0001752220?sid3=79b")
docs = loader.load()
content = docs[0].page_content
# Load the prompt
# prompt = hub.pull("langchain-ai/chain-of-density:4f55305e")


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 최근 스포츠 세계에서 발생한 중요한 사건에 대해 다루고 있습니다. 특히, 여러 스포츠 분야에서의 주요 경기 결과와 그에 따른 선수들의 성과, 팀의 순위 변동 등을 상세히 설명하고 있습니다. 또한, 이러한 경기들이 앞으로 스포츠계에 미칠 영향에 대해서도 분석하고 있으며, 특정 선수들의 인터뷰 내용을 인용하여 그들의 생각과 감정을 전달하고 있습니다. 이와 함께, 경기 일정, 선수들의 부상 소식, 이적 소식 등도 포함되어 있어 스포츠 팬들에게 유용한 정보를 제공하고 있습니다. 전반적으로, 이 기사는 스포츠에 대한 광범위한 정보를 제공하며, 독자들이 최신 스포츠 동향을 이해하는 데 도움을 주고자 합니다."
    },
    {
        "Missing_Entities": "경기 결과; 선수 인터뷰",
        "Denser_Summary": "이 기사는 다양한 스포츠 분야에서의 경기 결과와 선수들의 인터뷰 내용을 포함하여 스포츠 세계의 최신 동향을 다룹니다. 주요 경기의 결과와 선수들의 성과, 팀 순위 변동을 상세히 분석하고, 선수들의 인터뷰를 통해 그들의 생각과 감정을 전달합니다. 또한, 경기 일정, 부상 및 이적 소식 등을 포함하여 스포츠 팬들에게 유용한 정보를 제공합니다. 이 기사는 스포츠에 대한 광범위한 정보를 제공하며, 독자들이 최신 스포츠 동향을 이해하는 데 도움을 주는 것을 목표로 합니다."
    },
    {
        "Missing_Entities": "팀 순위 변동; 부상 및 이적 소식",
        "Denser_Summary": "이 기사는 다양한 스포츠 분야의 경기 결과, 선수 인터뷰, 팀 순위 변동, 부상 및 이적 소식을 포함하여 최신 스포츠 동향을 다룹니다. 경기 결과와 선수들의 성과 분석, 선수들의 생각과 감정을 담은 인터뷰, 팀 순위의 변동, 선수들의 부상과 이적 소식까